# Snow model
*R.A. Collenteur, University of Graz / Eawag, November 2021*

In this notebook it is shown how to account for snowfall and smowmelt on groundwater recharge and groundwater levels, using a degree-day snow model. This notebook is work in progress and will be extended in the future.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve

import pastas as ps

ps.set_log_level("ERROR")
ps.show_versions()

## 1. Load data
In this notebook we will look at some data for a well near Heby, Sweden. All the meteorological data is taken from the E-OBS database. As can be observed from the temperature time series, the temparature regularly drops below zero in winter. Given this observation, we may expect precipitation to (partially) fall as snow during these periods.

In [ ]:
head = pd.read_csv("data/heby_head.csv", index_col=0, parse_dates=True).squeeze()
evap = pd.read_csv("data/heby_evap.csv", index_col=0, parse_dates=True).squeeze()
prec = pd.read_csv("data/heby_prec.csv", index_col=0, parse_dates=True).squeeze()
temp = pd.read_csv("data/heby_temp.csv", index_col=0, parse_dates=True).squeeze()

ps.plots.series(head=head, stresses=[prec, evap, temp]);

## 2. Make a simple model
First we create a simple model with precipitation and potential evaporation as input, using the non-linear `FlexModel` to compute the recharge flux. We not not yet take snowfall into account, and thus assume all precipitation occurs as snowfall. The model is calibrated and the results are visualized for inspection.

In [ ]:
# Settings
tmin = "1985"  # Needs warmup
tmax = "2010"

In [ ]:
ml1 = ps.Model(head)
sm1 = ps.RechargeModel(
    prec, evap, recharge=ps.rch.FlexModel(), rfunc=ps.Gamma(), name="rch"
)
ml1.add_stressmodel(sm1)

# As the evaporation used is a very rough estimation, vary k_v
ml1.set_parameter("rch_kv", vary=True)

# Solve the Pastas model in two steps
ml1.solve(tmin=tmin, tmax=tmax, noise=False, fit_constant=False, report=False)
ml1.set_parameter("rch_srmax", vary=False)
ml1.solve(tmin=tmin, tmax=tmax, noise=True, fit_constant=False, initial=False)
ml1.plot(figsize=(10, 3));

The model fit with the data is not too bad, but we are clearly missing the highs and lows of the observed groundwater levels. This could have many causes, but in this case we may suspect that the occurence of snowfall and melt impacts the results.

## 3. Account for snowfall and snow melt

A second model is now created that accounts for snowfall and melt through a degree-day snow model (see e.g., Kavetski & Kuczera (2007). To run the model we add the keyword `snow=True` to the `FlexModel` and provide a time series of mean daily temperature to the `RechargeModel`. The temperature time series is used to split the precipitation into snowfall and rainfall.

In [ ]:
ml2 = ps.Model(head)
sm2 = ps.RechargeModel(
    prec,
    evap,
    recharge=ps.rch.FlexModel(snow=True),
    rfunc=ps.Gamma(),
    name="rch",
    temp=temp,
)
ml2.add_stressmodel(sm2)

# As the evaporation used is a very rough estimation, vary k_v
ml2.set_parameter("rch_kv", vary=True)

# Solve the Pastas model in two steps
ml2.solve(tmin=tmin, tmax=tmax, noise=False, fit_constant=False, report=False)
ml2.set_parameter("rch_srmax", vary=False)
ml2.solve(tmin=tmin, tmax=tmax, noise=True, fit_constant=False, initial=False)

## Compare results
From the `fit_report` we can already observe that the model fit improved quite a bit. We can also  visualize the results to see how the model improved.

In [ ]:
ax = ml2.plot(figsize=(10, 3))
ml1.simulate().plot(ax=ax)
plt.legend(
    [
        "Observations",
        "Model w Snow NSE={:.2f}".format(ml2.stats.nse()),
        "Model w/o Snow NSE={:.2f}".format(ml1.stats.nse()),
    ],
    ncol=3,
)

## Extract the water balance (States & Fluxes)

In [ ]:
df = ml2.stressmodels["rch"].get_water_balance(
    ml2.get_parameters("rch"), tmin=tmin, tmax=tmax
)
df.plot(subplots=True, figsize=(10, 10));

## References

- Kavetski, D. and Kuczera, G. (2007). Model smoothing strategies to remove microscale discontinuities and  spurious secondary optima  in  objective  functions  in  hydrological calibration. Water Resources Research, 43(3).